In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML
from datetime import datetime


In [50]:
import plotly
plotly.offline.init_notebook_mode(connected=True)
from plotly.graph_objs import *
from plotly import tools
import plotly.graph_objects as go
import seaborn as sns

In [51]:
arrivals_syr = pd.read_csv('arrivals.csv', date_parser=lambda x: datetime.strptime(x, '%m/%d/%y'))
arrivals_syr.head()

/var/folders/1r/26j1lw0x6xgg8lq3_h2b3nh80000gn/T/ipykernel_66956/2578234064.py:1: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.



,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Wheels-on Time,Taxi-In time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes)
0,UA,01/01/2022,"1,282.00",N4901U,IAD,23:10,00:01,70.00,76.00,51.00,23:55,6.00,23.00,0.00,6.00,0.00,22.00
1,UA,01/01/2023,604.00,N814UA,DEN,14:58,14:52,193.00,177.00,-6.00,14:48,4.00,0.00,0.00,0.00,0.00,0.00
2,UA,01/01/2023,"2,488.00",N38458,EWR,23:14,23:15,75.00,62.00,1.00,23:10,5.00,0.00,0.00,0.00,0.00,0.00
3,UA,01/01/2023,"2,645.00",N23721,ORD,23:57,23:47,107.00,100.00,-10.00,23:41,6.00,0.00,0.00,0.00,0.00,0.00
4,UA,01/02/2022,"1,282.00",N4901U,IAD,23:10,23:27,70.00,64.00,17.00,23:19,8.00,17.00,0.00,0.00,0.00,0.00


In [52]:
len(arrivals_syr)
arrivals_syr.columns
arrivals_syr.dtypes

69525

Index(['Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', 'Tail Number',
       'Origin Airport', 'Scheduled Arrival Time', 'Actual Arrival Time',
       'Scheduled Elapsed Time (Minutes)', 'Actual Elapsed Time (Minutes)',
       'Arrival Delay (Minutes)', 'Wheels-on Time', 'Taxi-In time (Minutes)',
       'Delay Carrier (Minutes)', 'Delay Weather (Minutes)',
       'Delay National Aviation System (Minutes)', 'Delay Security (Minutes)',
       'Delay Late Aircraft Arrival (Minutes)'],
      dtype='object')

Carrier Code                                 object
Date (MM/DD/YYYY)                            object
Flight Number                               float64
Tail Number                                  object
Origin Airport                               object
Scheduled Arrival Time                       object
Actual Arrival Time                          object
Scheduled Elapsed Time (Minutes)            float64
Actual Elapsed Time (Minutes)               float64
Arrival Delay (Minutes)                     float64
Wheels-on Time                               object
Taxi-In time (Minutes)                      float64
Delay Carrier (Minutes)                     float64
Delay Weather (Minutes)                     float64
Delay National Aviation System (Minutes)    float64
Delay Security (Minutes)                    float64
Delay Late Aircraft Arrival (Minutes)       float64
dtype: object

In [53]:
arrivals_syr.drop(columns=['Tail Number', 'Scheduled Elapsed Time (Minutes)', 'Actual Elapsed Time (Minutes)', 'Wheels-on Time', 'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)', 'Delay Weather (Minutes)', 'Delay National Aviation System (Minutes)', 'Delay Security (Minutes)', 'Delay Late Aircraft Arrival (Minutes)'], inplace=True)


In [54]:
len(arrivals_syr)
arrivals_syr.columns
arrivals_syr.dtypes

69525

Index(['Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', 'Origin Airport',
       'Scheduled Arrival Time', 'Actual Arrival Time',
       'Arrival Delay (Minutes)'],
      dtype='object')

Carrier Code                object
Date (MM/DD/YYYY)           object
Flight Number              float64
Origin Airport              object
Scheduled Arrival Time      object
Actual Arrival Time         object
Arrival Delay (Minutes)    float64
dtype: object

In [55]:
departures_syr = pd.read_csv('departures.csv', date_parser=lambda x: datetime.strptime(x, '%m/%d/%y'))
departures_syr.head()

/var/folders/1r/26j1lw0x6xgg8lq3_h2b3nh80000gn/T/ipykernel_66956/2100041682.py:1: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.



,Unnamed: 0,index,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Wheels-off time,Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes),Origin Airport
0,0,53,G4,01/02/2020,1737,241NV,SYR,06:30,06:29,176,167,-1,06:42,13,0,0,0,0,0,FLL
1,1,102,G4,01/03/2019,1720,226NV,SYR,06:50,06:49,182,167,-1,07:01,12,0,0,0,0,0,FLL
2,2,121,G4,01/03/2021,1744,245NV,SYR,16:25,17:40,178,168,75,17:53,13,0,0,0,0,65,FLL
3,3,137,G4,01/03/2022,962,222NV,SYR,06:15,06:15,176,175,0,06:30,15,0,0,0,0,0,FLL
4,4,152,G4,01/03/2023,523,234NV,SYR,06:30,10:34,182,170,244,10:51,17,232,0,0,0,0,FLL


In [56]:
len(departures_syr)
departures_syr.columns
departures_syr.dtypes

56542

Index(['Unnamed: 0', 'index', 'Carrier Code', 'Date (MM/DD/YYYY)',
       'Flight Number', 'Tail Number', 'Destination Airport',
       'Scheduled departure time', 'Actual departure time',
       'Scheduled elapsed time (Minutes)', 'Actual elapsed time (Minutes)',
       'Departure delay (Minutes)', 'Wheels-off time',
       'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)',
       'Delay Weather (Minutes)', 'Delay National Aviation System (Minutes)',
       'Delay Security (Minutes)', 'Delay Late Aircraft Arrival (Minutes)',
       'Origin Airport'],
      dtype='object')

Unnamed: 0                                   int64
index                                        int64
Carrier Code                                object
Date (MM/DD/YYYY)                           object
Flight Number                                int64
Tail Number                                 object
Destination Airport                         object
Scheduled departure time                    object
Actual departure time                       object
Scheduled elapsed time (Minutes)             int64
Actual elapsed time (Minutes)                int64
Departure delay (Minutes)                    int64
Wheels-off time                             object
Taxi-Out time (Minutes)                      int64
Delay Carrier (Minutes)                      int64
Delay Weather (Minutes)                      int64
Delay National Aviation System (Minutes)     int64
Delay Security (Minutes)                     int64
Delay Late Aircraft Arrival (Minutes)        int64
Origin Airport                 

In [57]:
departures_syr.drop(columns=['Tail Number', 'Scheduled Elapsed Time (Minutes)', 'Actual elapsed time (Minutes)', 'Wheels-off time', 'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)', 'Delay Weather (Minutes)', 'Delay National Aviation System (Minutes)', 'Delay Security (Minutes)', 'Delay Late Aircraft Arrival (Minutes)'], inplace=True, errors='ignore')


In [58]:
len(departures_syr)
departures_syr.columns
departures_syr.dtypes

56542

Index(['Unnamed: 0', 'index', 'Carrier Code', 'Date (MM/DD/YYYY)',
       'Flight Number', 'Destination Airport', 'Scheduled departure time',
       'Actual departure time', 'Scheduled elapsed time (Minutes)',
       'Departure delay (Minutes)', 'Origin Airport'],
      dtype='object')

Unnamed: 0                           int64
index                                int64
Carrier Code                        object
Date (MM/DD/YYYY)                   object
Flight Number                        int64
Destination Airport                 object
Scheduled departure time            object
Actual departure time               object
Scheduled elapsed time (Minutes)     int64
Departure delay (Minutes)            int64
Origin Airport                      object
dtype: object

In [59]:
# Assuming departures_syr and arrivals_syr are pandas DataFrames
import pandas as pd

# Renaming columns to clarify data after merge (if needed)
departures_syr.rename(columns={
    'Scheduled departure time': 'Scheduled departure time',
    'Actual departure time': 'Actual departure time',
    'Scheduled elapsed time (Minutes)': 'Scheduled departure elapsed time (Minutes)',
    'Departure delay (Minutes)': 'Departure delay (Minutes)',
    'Origin Airport': 'Departure Airport'
}, inplace=True)

arrivals_syr.rename(columns={
    'Scheduled Arrival Time': 'Scheduled arrival time',
    'Actual Arrival Time': 'Actual arrival time',
    'Arrival Delay (Minutes)': 'Arrival delay (Minutes)',
    'Origin Airport': 'Arrival Origin Airport'
}, inplace=True)

# Merging datasets on 'Carrier Code', 'Date (MM/DD/YYYY)', and 'Flight Number'
merged_flights = pd.merge(departures_syr, arrivals_syr, 
                          on=['Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number'], 
                          how='outer', 
                          suffixes=('_depart', '_arrive'))

# The result is a DataFrame with both departure and arrival information for flights.


In [60]:
merged_flights.head()

,Unnamed: 0,index,Carrier Code,Date (MM/DD/YYYY),Flight Number,Destination Airport,Scheduled departure time,Actual departure time,Scheduled departure elapsed time (Minutes),Departure delay (Minutes),Departure Airport,Arrival Origin Airport,Scheduled arrival time,Actual arrival time,Arrival delay (Minutes)
0,0.00,53.00,G4,01/02/2020,"1,737.00",SYR,06:30,06:29,176.00,-1.00,FLL,FLL,09:26,09:16,-10.00
1,1.00,102.00,G4,01/03/2019,"1,720.00",SYR,06:50,06:49,182.00,-1.00,FLL,FLL,09:52,09:36,-16.00
2,2.00,121.00,G4,01/03/2021,"1,744.00",SYR,16:25,17:40,178.00,75.00,FLL,FLL,19:23,20:28,65.00
3,3.00,137.00,G4,01/03/2022,962.00,SYR,06:15,06:15,176.00,0.00,FLL,FLL,09:11,09:10,-1.00
4,4.00,152.00,G4,01/03/2023,523.00,SYR,06:30,10:34,182.00,244.00,FLL,FLL,09:32,13:24,232.00


In [61]:
# Dropping the 'Destination Airport' column from the merged DataFrame
merged_flights = merged_flights.drop(columns=['Destination Airport'])


,Unnamed: 0,index,Carrier Code,Date (MM/DD/YYYY),Flight Number,Scheduled departure time,Actual departure time,Scheduled departure elapsed time (Minutes),Departure delay (Minutes),Departure Airport,Arrival Origin Airport,Scheduled arrival time,Actual arrival time,Arrival delay (Minutes)
0,0.00,53.00,G4,01/02/2020,"1,737.00",06:30,06:29,176.00,-1.00,FLL,FLL,09:26,09:16,-10.00
1,1.00,102.00,G4,01/03/2019,"1,720.00",06:50,06:49,182.00,-1.00,FLL,FLL,09:52,09:36,-16.00
2,2.00,121.00,G4,01/03/2021,"1,744.00",16:25,17:40,178.00,75.00,FLL,FLL,19:23,20:28,65.00
3,3.00,137.00,G4,01/03/2022,962.00,06:15,06:15,176.00,0.00,FLL,FLL,09:11,09:10,-1.00
4,4.00,152.00,G4,01/03/2023,523.00,06:30,10:34,182.00,244.00,FLL,FLL,09:32,13:24,232.00


In [62]:

merged_flights.head()

,Unnamed: 0,index,Carrier Code,Date (MM/DD/YYYY),Flight Number,Scheduled departure time,Actual departure time,Scheduled departure elapsed time (Minutes),Departure delay (Minutes),Departure Airport,Arrival Origin Airport,Scheduled arrival time,Actual arrival time,Arrival delay (Minutes)
0,0.00,53.00,G4,01/02/2020,"1,737.00",06:30,06:29,176.00,-1.00,FLL,FLL,09:26,09:16,-10.00
1,1.00,102.00,G4,01/03/2019,"1,720.00",06:50,06:49,182.00,-1.00,FLL,FLL,09:52,09:36,-16.00
2,2.00,121.00,G4,01/03/2021,"1,744.00",16:25,17:40,178.00,75.00,FLL,FLL,19:23,20:28,65.00
3,3.00,137.00,G4,01/03/2022,962.00,06:15,06:15,176.00,0.00,FLL,FLL,09:11,09:10,-1.00
4,4.00,152.00,G4,01/03/2023,523.00,06:30,10:34,182.00,244.00,FLL,FLL,09:32,13:24,232.00


In [63]:
merged_flights.columns

Index(['Unnamed: 0', 'index', 'Carrier Code', 'Date (MM/DD/YYYY)',
       'Flight Number', 'Scheduled departure time', 'Actual departure time',
       'Scheduled departure elapsed time (Minutes)',
       'Departure delay (Minutes)', 'Departure Airport',
       'Arrival Origin Airport', 'Scheduled arrival time',
       'Actual arrival time', 'Arrival delay (Minutes)'],
      dtype='object')

In [64]:
from datetime import datetime

# Convert 'Date (MM/DD/YYYY)' to datetime
merged_flights['Date (MM/DD/YYYY)'] = pd.to_datetime(merged_flights['Date (MM/DD/YYYY)'])

# Extract day of the week as a categorical feature
merged_flights['Day of Week'] = merged_flights['Date (MM/DD/YYYY)'].dt.day_name()

# Optionally, convert times to parts of the day or other meaningful categories


In [66]:
merged_flights.dtypes
merged_flights.head()

Unnamed: 0                                           float64
index                                                float64
Carrier Code                                          object
Date (MM/DD/YYYY)                             datetime64[ns]
Flight Number                                        float64
Scheduled departure time                              object
Actual departure time                                 object
Scheduled departure elapsed time (Minutes)           float64
Departure delay (Minutes)                            float64
Departure Airport                                     object
Arrival Origin Airport                                object
Scheduled arrival time                                object
Actual arrival time                                   object
Arrival delay (Minutes)                              float64
Day of Week                                           object
dtype: object

,Unnamed: 0,index,Carrier Code,Date (MM/DD/YYYY),Flight Number,Scheduled departure time,Actual departure time,Scheduled departure elapsed time (Minutes),Departure delay (Minutes),Departure Airport,Arrival Origin Airport,Scheduled arrival time,Actual arrival time,Arrival delay (Minutes),Day of Week
0,0.00,53.00,G4,2020-01-02,"1,737.00",06:30,06:29,176.00,-1.00,FLL,FLL,09:26,09:16,-10.00,Thursday
1,1.00,102.00,G4,2019-01-03,"1,720.00",06:50,06:49,182.00,-1.00,FLL,FLL,09:52,09:36,-16.00,Thursday
2,2.00,121.00,G4,2021-01-03,"1,744.00",16:25,17:40,178.00,75.00,FLL,FLL,19:23,20:28,65.00,Sunday
3,3.00,137.00,G4,2022-01-03,962.00,06:15,06:15,176.00,0.00,FLL,FLL,09:11,09:10,-1.00,Monday
4,4.00,152.00,G4,2023-01-03,523.00,06:30,10:34,182.00,244.00,FLL,FLL,09:32,13:24,232.00,Tuesday


In [69]:
import pandas as pd

# Assuming your DataFrame is named 'merged_flights'
# List of categorical variables identified earlier
categorical_vars = ['Carrier Code', 'Flight Number', 'Departure Airport', 'Arrival Origin Airport', 'Day of Week']  # Add 'Day of Week' if you've created it from the date

# Apply one-hot encoding
merged_flights_encoded = pd.get_dummies(merged_flights, columns=categorical_vars)

# Check the new columns
print(merged_flights_encoded.columns)


Index(['Unnamed: 0', 'index', 'Date (MM/DD/YYYY)', 'Scheduled departure time',
       'Actual departure time', 'Scheduled departure elapsed time (Minutes)',
       'Departure delay (Minutes)', 'Scheduled arrival time',
       'Actual arrival time', 'Arrival delay (Minutes)',
       ...
       'Arrival Origin Airport_SFB', 'Arrival Origin Airport_SRQ',
       'Arrival Origin Airport_TPA', 'Day of Week_Friday',
       'Day of Week_Monday', 'Day of Week_Saturday', 'Day of Week_Sunday',
       'Day of Week_Thursday', 'Day of Week_Tuesday', 'Day of Week_Wednesday'],
      dtype='object', length=2007)


In [70]:
merged_flights_encoded.dtypes
merged_flights_encoded.head()

Unnamed: 0                         float64
index                              float64
Date (MM/DD/YYYY)           datetime64[ns]
Scheduled departure time            object
Actual departure time               object
                                 ...      
Day of Week_Saturday                  bool
Day of Week_Sunday                    bool
Day of Week_Thursday                  bool
Day of Week_Tuesday                   bool
Day of Week_Wednesday                 bool
Length: 2007, dtype: object

,Unnamed: 0,index,Date (MM/DD/YYYY),Scheduled departure time,Actual departure time,Scheduled departure elapsed time (Minutes),Departure delay (Minutes),Scheduled arrival time,Actual arrival time,Arrival delay (Minutes),...,Arrival Origin Airport_SFB,Arrival Origin Airport_SRQ,Arrival Origin Airport_TPA,Day of Week_Friday,Day of Week_Monday,Day of Week_Saturday,Day of Week_Sunday,Day of Week_Thursday,Day of Week_Tuesday,Day of Week_Wednesday
0,0.00,53.00,2020-01-02,06:30,06:29,176.00,-1.00,09:26,09:16,-10.00,...,False,False,False,False,False,False,False,True,False,False
1,1.00,102.00,2019-01-03,06:50,06:49,182.00,-1.00,09:52,09:36,-16.00,...,False,False,False,False,False,False,False,True,False,False
2,2.00,121.00,2021-01-03,16:25,17:40,178.00,75.00,19:23,20:28,65.00,...,False,False,False,False,False,False,True,False,False,False
3,3.00,137.00,2022-01-03,06:15,06:15,176.00,0.00,09:11,09:10,-1.00,...,False,False,False,False,True,False,False,False,False,False
4,4.00,152.00,2023-01-03,06:30,10:34,182.00,244.00,09:32,13:24,232.00,...,False,False,False,False,False,False,False,False,True,False
